# Ottenimento prossimi match
Lo script deve permettere di prendere i prossimi 10 match. 
Verrà fatta una predizione su questi match, prima deve essere calcolata la media di ciascun match che è data dai 5 match precedenti.

In [1]:
from analysis import MatchAnalysis
import pandas as pd
import util_strings as utils
from ranking import Ranking
from football_predictions import FootballPredictions

In [2]:
championship = pd.read_csv(utils.championship_actual_season, index_col=0)
season = utils.championship_actual_season.replace('.csv', '')[-9:]

championship['date'] = pd.to_datetime(championship['date'], format='%Y-%m-%d')

championship.result = championship.result.astype(str)
new_matches = championship[championship.result == 'nan']

In [3]:
matches = new_matches[:10][['team1', 'team2', 'date', 'result']]
matches['season'] = season
matches.rename(columns= {'team1': 'home', 'team2': 'away'}, inplace=True)
matches.reset_index(drop=True, inplace=True)
matches.home = matches.home.str.lower()
matches.away = matches.away.str.lower()

matches.to_csv(utils.next_matches)

## Ottenimento testo FootballPrediction
Scaricamento del testo di ciascuna partita

In [4]:
fp = FootballPredictions(matches=matches)
fp.update_originalNotation()
fp.get_urls()
fp.get_predictions(utils.next_matches, False)
fp.df['season'] = season
fp.recovery_games()
fp.matches_not_found()
fp.fix_dates(utils.next_matches)

100%|██████████| 4/4 [00:02<00:00,  1.89it/s]


In [5]:
from my_tokenizer import MyTokenizer

mt = MyTokenizer(fp.df)
mt.clean_text()
mt.open_vectorizer(utils.vectorizer)

In [6]:
import pickle

dt = pickle.load(open(utils.classificator_FP, 'rb'))
predictions = dt.predict(mt.vectorizer.transform(mt.cleaned_corpus))

Aggiungo le predizioni al dataset dei match

In [7]:
cont = 0
for i, match in matches.iterrows():
    matches.at[i, 'prediction'] = predictions[cont]
    cont += 1

## Calcolo delle medie
Per ciascun match devo calcolare le statistiche date dalle medie dei 5 match precedenti e poi devo ottenere la predizione di football_prediction.
Sostituisco inoltre i codici delle squadre

Per calcolare le medie devo prendere tutti i match di ciascuna squadra

In [8]:
ranking = Ranking('SerieA', '2022-2023')
ranking.read_matches(seasons=6, path=utils.ranking)

ma = MatchAnalysis()
ma.set_ranking(ranking=ranking)
#leggo tutti i match (doppi perché ci sono le statistiche x ogni squadra)
ma.read_matches(utils.merged_statistics)
#creo una lista di oggetti (lista di squadre)
ma.create_team_dataset() 

In [12]:
matches['date'] = pd.to_datetime(matches['date'], format='%Y-%m-%d') 
float_features_and_avg = ['goals','total_shots','shots_on_target','goals_on_penalty','total_penalties','percentage_possession','fouls','yellow_cards','red_cards']

import numpy as np
for f in float_features_and_avg:
    matches[f] = np.nan

matches[float_features_and_avg] = matches[float_features_and_avg].astype(float)

label = {'N': 0, 'V': 1, 'P':2}

ma.diff_dataset = matches.copy()
for i, match in ma.diff_dataset.iterrows(): 
    match.season = '2022-2023'
    avgH, c = ma.get_team_by_name(match.home).get_avg_last_X_matches(5, match.date, float_features_and_avg)
    avgA, c = ma.get_team_by_name(match.away).get_avg_last_X_matches(5, match.date, float_features_and_avg)
    
    ma.get_ranks(match, i)

    for col in float_features_and_avg:
        diff = avgH[col] - avgA[col]
        ma.diff_dataset.at[i, col] = diff
    
    ma.diff_dataset.at[i, 'prediction'] = label[match.prediction]

    ma.diff_dataset.at[i, 'home'] = ma.get_id_by_name(match.home)
    ma.diff_dataset.at[i, 'away'] = ma.get_id_by_name(match.away)

ma.diff_dataset.drop(columns=['result', 'date', 'season'], inplace=True)

In [13]:
ma.diff_dataset
dt = pickle.load(open(utils.classificator, 'rb'))
predictions = dt.predict(ma.diff_dataset)

In [14]:
df_predictions = pd.DataFrame()
for i, match in matches.iterrows():
    match['prediction'] = predictions[i]
    df_predictions = df_predictions.append(match[['home','away','prediction']])

In [15]:
df_predictions

,home,away,prediction
0,juventus,empoli,1.0
1,salernitana,spezia,1.0
2,fiorentina,inter,2.0
3,milan,monza,1.0
4,atalanta,lazio,2.0
5,bologna,lecce,1.0
6,roma,napoli,2.0
7,udinese,torino,1.0
8,sassuolo,hellas verona,0.0
9,cremonese,sampdoria,1.0
